In [1]:
# Importing Library
from bs4 import BeautifulSoup
import requests
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np
import re

In [2]:
URL = "https://health.usnews.com/doctors/city-index/new-jersey"

In [3]:
headers = {'User-Agent': 'Chrome/68.0.3440.106'} # To encounter 403 error

In [4]:
page = requests.get(URL, headers=headers)

In [5]:
#check if the page was downloaded - status code should be 200
page.status_code

200

In [6]:
#Parse the HTML content from page using Beautiful Soup
soup=BeautifulSoup(page.content,'html.parser')
#soup = BeautifulSoup(page.text)

# Links of all the location in New Jersey (Page - 2)

In [7]:
a = soup.find_all('li',class_= 'List__ListItem-xbc1kt-1 kkayKp')

In [8]:
location_links=[]
for i in a:
    b = i.find('a')
    location_links.append(b.get('href'))

In [9]:
location_links[0]

'/doctors/specialists-index/new-jersey/absecon'

In [10]:
len(location_links)

588

In [11]:
location_links[0]

'/doctors/specialists-index/new-jersey/absecon'

In [12]:
page2 = []
for i in location_links:
    url = ("https://health.usnews.com"+ str(i))
    page2.append(url)

In [13]:
len(page2)

588

In [14]:
page2[0]

'https://health.usnews.com/doctors/specialists-index/new-jersey/absecon'

# Doctors specialize in a particular area of medicine (Page - 3)

In [15]:
# For city Absecon
city1 = page2[0]

In [16]:
page_2 = requests.get(city1, headers=headers)

In [17]:
page_2.status_code

200

In [18]:
soup_2 = BeautifulSoup(page_2.content,'html.parser')

In [19]:
a = soup_2.find_all('li',class_= 'List__ListItem-xbc1kt-1 kkayKp')

In [20]:
doc_spec=[]
for i in a:
    b = i.find('a')
    doc_spec.append(b.get('href'))

In [21]:
doc_spec[0]

'/doctors/anesthesiologists/new-jersey/absecon'

In [22]:
page3 = []
for i in doc_spec:
    url = ("https://health.usnews.com"+ str(i))
    page3.append(url)

In [23]:
page3[0]

'https://health.usnews.com/doctors/anesthesiologists/new-jersey/absecon'

In [24]:
len(page3)

116

# Anesthesiologists (Page - 3)

In [25]:
doc = page3[0]

In [26]:
doc

'https://health.usnews.com/doctors/anesthesiologists/new-jersey/absecon'

In [27]:
page_3 = requests.get(doc, headers=headers)

In [28]:
page_3.status_code

200

In [29]:
soup_3 = BeautifulSoup(page_3.content,'html.parser')

In [30]:
a = soup_3.find_all('a',class_= 'search-result-link bar-tighter')

In [31]:
doc_links=[]
for i in a:
    doc_links.append(i.get('href'))

In [32]:
doc_links[0]

'/doctors/kenneth-abrams-406792'

In [33]:
page4 = []
for i in doc_links:
    url = ("https://health.usnews.com"+ str(i))
    page4.append(url)

In [34]:
page4[0]

'https://health.usnews.com/doctors/kenneth-abrams-406792'

In [35]:
len(page4)

20

# Scrap the data from the website

In [36]:
overview=[]
name=[]
year_in_practice=[]
language=[]
location=[]
affiliation=[]
specialties=[]
sub_specialties=[]
education_training=[]

for i in page4:
    page_4 = requests.get(i, headers=headers)
    soup_4=BeautifulSoup(page_4.content,'html.parser')
    #OverView
    overview.append(str(soup_4.find('div',{'class' :'block-normal clearfix'}).getText()))
    
    #Name Of the Doctor
    name.append(str(soup_4.find('h1',{'class':'hero-heading flex-media-heading block-tight doctor-name '}).getText()))
    
    #Years In Practice
    year_in_practice.append(soup_4.find_all('span',class_='text-large heading-normal-for-small-only right-for-medium-up')[1].getText())
    
    #Language
    language.append(soup_4.find('span',class_='text-large heading-normal-for-small-only right-for-medium-up text-right showmore').getText())
    
    #Location
    location.append(soup_4.find('span',attrs={'data-js-id':'doctor-address'}).getText())
    
    #Affiliation
    affiliation.append(soup_4.find('p',class_='text-muted').getText())
    
    #Specialties
    specialties.append(soup_4.find('a',class_='text-large').getText())
    
    #sub_specialties
    s_speciality = soup_4.find_all('p',class_='text-large block-tight')
    s = []
    for i in range(len(s_speciality)):
        s.append(s_speciality[i].get_text())
    sub_specialties.append(s)
    
    #Education   
    education = soup_4.findAll('section',class_='block-loosest')
    for i in range(len(education)):
        try:
            if education[i].find('h2',class_="heading-larger block-normal").get_text() == "Education & Medical Training":
                a = education[i].find_all('li',class_ = 'block-tight')
                edu_lst = []
                for j in range(len(a)):
                    edu_lst.append(education[i].find_all('li',class_ = 'block-tight')[j].get_text())
                education_training.append(edu_lst)
                break
        except:
            education_training.append('NaN')

In [37]:
Certification_licensure=[]
for i in page4:
    page_4 = requests.get(i, headers=headers)
    soup_4=BeautifulSoup(page_4.content,'html.parser') 
    #Education   
    education = soup_4.findAll('section',class_='block-loosest')
    for i in range(len(education)):
        try:
            if education[i].find('h2',class_="heading-larger block-normal").get_text() == "Certifications & Licensure":
                b = education[i].find_all('li',class_ = 'block-tight')
                cet = []
                for j in range(len(b)):
                    cet.append(education[i].find_all('li',class_ = 'block-tight')[j].get_text())
                Certification_licensure.append(cet)
                break
        except:
            Certification_licensure.append('NaN')

# Creating DataFrame & Cleaning

In [38]:
data = pd.DataFrame(columns = ['overview','name','year_in_practice','language','location','affiliation','specialties','sub_specialties','education_training','Certification_licensure'])

In [39]:
data

,overview,name,year_in_practice,language,location,affiliation,specialties,sub_specialties,education_training,Certification_licensure


In [40]:
data['overview'] = overview
data['name'] = name
data['year_in_practice'] = year_in_practice
data['language'] = language
data['location'] = location
data['affiliation'] = affiliation
data['specialties'] = specialties
data['sub_specialties'] = sub_specialties
data['education_training'] = education_training
data['Certification_licensure']=Certification_licensure

In [41]:
data1 = data.replace('\n','', regex=True)

In [42]:
data1 = data1.replace("\t", "")

In [43]:
for i in range(len(data1)):
    for j in range(len(data1.education_training[i])):
        data1.education_training[i][j].strip()

In [44]:
for i in range(len(data1)):
    for j in range(len(Certification_licensure[i])):
        data1.Certification_licensure[i][j].strip()

In [45]:
data1.overview = data1.overview.str.strip()
data1.name = data1.name.str.strip()
data1.location = data1.location.str.strip()

In [46]:
data1

,overview,name,year_in_practice,language,location,affiliation,specialties,sub_specialties,education_training,Certification_licensure
0,Dr. Kenneth Abrams is an anesthesiologist in P...,Dr. Kenneth Abrams ...,21+,English,65 Jimmie Leeds Road Department...,Dr. Abrams does not have any hospital affiliat...,Anesthesiology,[General Anesthesiology],[\n Albert Einstein College of Medi...,[\n American Board of Anesthesiolog...
1,Dr. Mohammad Chohan is an anesthesiologist in ...,Dr. Mohammad Chohan ...,21+,English,"65 Jimmie Leeds Road Pomona, NJ...",Dr. Chohan does not have any insurances listed.,Anesthesiology,[General Anesthesiology],[\n George Washington University\n ...,[\n American Board of Anesthesiolog...
2,Dr. Olawale Fadugba is an anesthesiologist in ...,Dr. Olawale Fadugba ...,21+,English,65 Jimmie Leeds Road Department...,Dr. Fadugba does not have any insurances listed.,Anesthesiology,[General Anesthesiology],[\n New York Medical College\n ...,[\n American Board of Anesthesiolog...
3,Dr. Nicholas Incandela is an anesthesiologist ...,Dr. Nicholas Incandela ...,21+,English,65 Jimmie Leeds Rd Dept Of Anes...,Dr. Incandela does not have any hospital affil...,Anesthesiology,[General Anesthesiology],[\n Albert Einstein College of Medi...,[\n American Board of Anesthesiolog...
4,Dr. Merceditas Lagmay is an anesthesiologist i...,Dr. Merceditas Lagmay ...,1 - 2,English,"65 Jimmie Leeds Road Pomona, NJ...",Dr. Lagmay does not have any insurances listed.,Anesthesiology,[General Anesthesiology],[\n Mount Sinai School of Medicine\...,[\n American Board of Anesthesiolog...
5,Dr. Thomas Long is an anesthesiologist in Pomo...,Dr. Thomas Long ...,21+,English,"65 Jimmie Leeds Road Pomona, NJ...",Dr. Long does not have any insurances listed.,Anesthesiology,"[General Anesthesiology, Critical Care Medicine]",[\n Albert Einstein College of Medi...,[\n American Board of Anesthesiolog...
6,Dr. Denise Marable is an anesthesiologist in P...,Dr. Denise Marable ...,21+,English,"65 Jimmie Leeds Road Pomona, NJ...",Dr. Marable does not have any insurances listed.,Anesthesiology,[General Anesthesiology],[\n Detroit Medical Center - Wayne ...,[\n American Board of Anesthesiolog...
7,Dr. Sudha Shah is an anesthesiologist in Pomon...,Dr. Sudha Shah ...,21+,English,"65 Jimmie Leeds Road Pomona, NJ...",Dr. Shah does not have any insurances listed.,Anesthesiology,[General Anesthesiology],[\n Albert Einstein Healthcare Netw...,[\n American Board of Anesthesiolog...
8,"Dr. Qin Wang is an anesthesiologist in Pomona,...",Dr. Qin Wang ...,21+,English,"65 Jimmie Leeds Road Pomona, NJ...",Dr. Wang does not have any insurances listed.,Anesthesiology,[General Anesthesiology],[\n Wannan Medical College\n ...,[\n American Board of Anesthesiolog...
9,Dr. Su-Lin Yao is an anesthesiologist in Pomon...,Dr. Su-Lin Yao ...,1 - 2,English,"65 Jimmie Leeds Road Pomona, NJ...",Dr. Yao does not have any insurances listed.,Anesthesiology,[General Anesthesiology],NaN,[\n American Board of Anesthesiolog...


In [47]:
data1.to_csv('data_New-Jersey.csv')

# Elasticsearch & Python 

In [48]:
import csv
from elasticsearch import Elasticsearch,helpers

In [49]:
es = Elasticsearch(HOST="http://localhost",PORT=9200)

In [50]:
es.indices.create(index='first_index',ignore=400)

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [first_index/0J7zr8_CReO1WZpegZh5Yg] already exists',
    'index_uuid': '0J7zr8_CReO1WZpegZh5Yg',
    'index': 'first_index'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [first_index/0J7zr8_CReO1WZpegZh5Yg] already exists',
  'index_uuid': '0J7zr8_CReO1WZpegZh5Yg',
  'index': 'first_index'},
 'status': 400}

In [51]:
es.indices.exists(index="first_index")

True

In [52]:
es.info

<bound method Elasticsearch.info of <Elasticsearch([{}])>>

In [53]:
#with open('data.csv') as a:
    #reader = csv.DictReader(a)
    #helpers.bulk(es, reader,index='my-index', doc_type='my-type')